In [1]:
from Bio.PDB import PDBParser, PDBIO, internal_coords
import numpy as np

In [7]:
parser = PDBParser()
structure = parser.get_structure("5IRE", "5ire.pdb")

In [17]:
model = structure.child_list[0]

In [19]:
chain = model.child_list[0]

In [21]:
residue = chain.child_list[0]

In [69]:
atom = residue.child_list[5]
print(atom.name + " " + str(atom.get_serial_number()))

CG1 6


In [48]:
internal_coords.

('5IRE', 0, 'A', (' ', 1, ' '), ('N', ' '))


In [80]:
test = np.array([1, 2, 2])
test2 = np.array([2, 3, 4])
print(np.linalg.norm(test))

3.0


In [103]:
def dihedral(a, b, c, d):
    # calculating vectors in plane 1
    ba = a - b
    bc = c - b
    # calculating normal vector of plane 1
    n1 = np.cross(ba, bc)
    print(n1)

    # vectors in plane 2
    cb = b - c
    cd = d - c
    # normal vector of plane 2
    n2 = np.cross(cb, cd)
    print(n2)

    # angle between both planes:
    dot_ab = np.dot(n1, n2)
    norm_ab = np.linalg.norm(n1)*np.linalg.norm(n2)
    alpha = np.arccos(dot_ab/norm_ab)

    return alpha*180/np.pi


In [16]:
def dihedral2(a, b, c, d):
    # calculating vectors in planes
    ab = b - a
    bc = c - b
    cd = d - c

    # calculating normal vectors n1 and n2 of planes:
    n1 = np.cross(ab, bc)
    n2 = np.cross(bc, cd)
    print("n1: " + str(n1))
    print("n2: " + str(n2))

    # cos(thetha) for planes ab bc and bc cd
    cos_theta = (np.dot(n1, n2))/(np.linalg.norm(n1)*np.linalg.norm(n2))
    print("cos theta: " + str(cos_theta))

    # sin(thetha) for the same planes
    sin_theta = np.dot(np.cross(n1, n2), bc)/(np.linalg.norm(n1)*np.linalg.norm(n2)*np.linalg.norm(bc))
    print("sin theta: " + str(sin_theta))

    # calculating theta
    theta = np.arctan2(sin_theta, cos_theta)

    return theta*180/np.pi

In [34]:
a = np.array([0, 1, 0])
b = np.array([0, 0, 0])
c = np.array([1, 0, 0])
d = np.array([1, -1, -0.1])
print(dihedral2(a, b, c, d))

n1: [0 0 1]
n2: [ 0.   0.1 -1. ]
cos theta: -0.9950371902099893
sin theta: -0.09950371902099893
-174.28940686250036
